<a href="https://colab.research.google.com/github/EP111111/Doc_analysis/blob/main/docs_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()


In [ ]:
!pip install shapely==1.8.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.9 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: Shapely 1.8.5.post1
    Uninstalling Shapely-1.8.5.post1:
      Successfully uninstalled Shapely-1.8.5.post1


In [ ]:

!pip install google-cloud-aiplatform --upgrade
!pip install textract
!pip install python-docx

In [ ]:
import vertexai
from vertexai.language_models import TextGenerationModel
from google.colab import drive
import textract
import numpy as np
import pandas as pd
import re


\

In [ ]:
PROJECT_ID = "ics-analysis-dev"
vertexai.init(project=PROJECT_ID, location="us-central1")

model = TextGenerationModel.from_pretrained("text-bison@001")

In [ ]:
from functions import *
import docx

In [ ]:
def read_word_file(file_path):
    doc = docx.Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text


document_text = read_word_file("Project Alpha fixed.docx")



#Solution 1:
Split document into several sections by Large Language Model



In [ ]:
def split_document(document, max_section_length=10000):
    sections = {}
    current_section = ""
    section_key = 1

    paragraphs = document.split("\n")

    for paragraph in paragraphs:
        if len(current_section) + len(paragraph) <= max_section_length:
            current_section += paragraph + "\n"
        else:

            sections[section_key] = current_section
            current_section = paragraph + "\n"
            section_key += 1


    sections[section_key] = current_section

    return sections



## Seperating docs into sections
Palm2 has a limit of 8000 tokens
if the section is long, we will have less sections, then the model will be faster.
But if the section is too long, the output will exceed the maxmium output size which is 1024 token
if the section is short, we will call palm2 more times. it takes longer time

In [ ]:
LLM_c= LLM_SPLIT_FUNC(document_text,model,12000)
LLM_c.keys()

dict_keys([0, 1, 2])

In [ ]:
length=0
for i in LLM_c.values():
  print(len(i))
  length+=len(i)
print(f'total length： {length}')
print(f'document length： {len(document_text)}')

10464
8485
10184
total length： 29133
document length： 29133


In [ ]:
#print seperator
for i in LLM_c.values():
  if i.split('\n')[0]=='':
    print(i.split('\n')[1])
  else:
    print(i.split('\n')[0])


The Energy Program at AGENTS is there to assist members of the community with their heating costs throughout the winter as well as home electricity needs. This is a program that relies heavily on specific state and federal regulations and funding sources, so it must be managed and reconfigured on a yearly basis as the regulations and funding sources change.
VENDOR INFORMATION


In [ ]:
#generate questions for each section
questions={}
for ind, i in enumerate(LLM_c.keys()):
  a=Query_Vertex('take questions out from the provided article, if no question in the article, return "no". ' ,LLM_c[i],model).split('\n')
  if len(a)!=1:
    print(i)
    print(a)
    questions[i]=a

0
['Are there any specific requirements for the software?', 'What are the technical specifications of the system?', 'What are the reporting requirements?', 'What are the data security requirements?', 'What are the accounting requirements?']
1
['Provide an overview of your company, including date of incorporation, operating locations, number of employees, area(s) of specialty and number of successful projects with Non-Profit / Community Action Program customers.', 'Are you a woman or minority owned business? Please indicate in your response if you are.', 'Provide information about the specification of the project organizations with which you have experience (e.g., onshore, offshore, onshore PM, offshore dedicated development team, etc.).', 'Describe how you screen and hire staff. What level of skill and certifications, if any, must they have? How do you evaluate their experience and performance? What was your staff turnover rate throughout the preceding three (3) years for each year? Do

In [ ]:
import time
def take_context(question,cont):
  context=empty_Vertex(f'''
  Document:{cont}
  ___above this line is the document,below this line is what you need to do___
  EXTRACT context related to the question. The output must be from the above Document, if no context related to the input exist, return:"no" only

  question:["{question}"]
  output:context OR "no"
  ''')
  return context

#Solution 2:
Separate document into several sections by main Headings

In [ ]:
headings = read_word_file("headings.docx")

In [ ]:
print(headings)

In [ ]:
a=Query_Vertex('This is a table of content, please extract main heading1' ,headings,model)
b=re.sub('  ','',a)
b=re.sub('\d. ','',b)
main_table=re.sub('\d.','',b).split('\n')
print(main_table)

['\tTERMS & CONDITIONS', '\tGENERAL INFORMATION', '\tTECHNICAL SPECIFICATIONS', '\tHIGH-LEVEL PROJECT REQUIREMENTS', '\tVENDOR INFORMATION', '\tVENDOR PROPOSAL', '\tMETHOD OF EVALUATIONS AND AWARD']


In [ ]:
main_table=['INTRODUCTION',
 'TERMS & CONDITIONS',
 'GENERAL INFORMATION',
 'TECHNICAL SPECIFICATIONS',
 'HIGH-LEVEL PROJECT REQUIREMENTS',
 'VENDOR INFORMATION',
 'VENDOR PROPOSAL',
 'METHOD OF EVALUATIONS AND AWARD']

In [ ]:
split_C = []
split_c = {}

for i in range(len(main_table)):
    try:
        if i < len(main_table) - 1:
            PART = document_text.split(main_table[i])[1].split(main_table[i + 1])[0]
        else:
            PART = document_text.split(main_table[i])[-1]
        split_C.append(PART)
        split_c[main_table[i]] = PART
    except IndexError:
        # Handle the case where the delimiter in main_table cannot be found
        # Print an error message or take appropriate action
        print(f"Delimiter '{main_table[i]}' not found in document_text.")


In [ ]:
for a in split_c.values():
  print(len(a))

724
632
4555
10042
2890
3517
2840
3763


In [ ]:
questions={}
for ind, i in enumerate(split_c.keys()):
  a=Query_Vertex('only extract questions from the above article, if no question in the article, return "no". ' ,split_c[i],model).split('\n')
  if len(a)!=1:
    print(i)
    print(a)
    questions[i]=a

VENDOR INFORMATION
['Provide an overview of your company, including date of incorporation, operating locations, number of employees, area(s) of specialty and number of successful projects with Non-Profit / Community Action Program customers.', 'Are you a woman or minority owned business? Please indicate in your response if you are.', 'Provide information about the specification of the project organizations with which you have experience (e.g., onshore, offshore, onshore PM, offshore dedicated development team, etc.).', 'Describe how you screen and hire staff. What level of skill and certifications, if any, must they have? How do you evaluate their experience and performance? What was your staff turnover rate throughout the preceding three (3) years for each year? Do you require background checks on all employees? How often is a background check updated on existing employees? Do you employ any overseas resources?', '', 'Describe the Software Development Life Cycle used for customization